<a href="https://colab.research.google.com/github/gautham-umasankar/Quantum_COVID/blob/master/Classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Model Training



##DWave Installation

In [ ]:
!pip install dwave-ocean-sdk


     |████████████████████████████████| 399kB 3.6MB/s 
     |████████████████████████████████| 2.3MB 40.8MB/s 
     |████████████████████████████████| 92kB 9.1MB/s 
     |████████████████████████████████| 92kB 9.4MB/s 
     |████████████████████████████████| 92kB 9.5MB/s 
     |████████████████████████████████| 204kB 39.3MB/s 
     |████████████████████████████████| 4.7MB 20.1MB/s 
     |████████████████████████████████| 81kB 7.6MB/s 
     |████████████████████████████████| 798kB 45.5MB/s 
     |████████████████████████████████| 32.7MB 120kB/s 
  Created wheel for pyqubo: filename=pyqubo-0.4.0-cp36-none-any.whl size=40712 sha256=36488bde03b3aa361283293a2fd66c7f754f30330177fc8913ab093bfa75b3d1
  Stored in directory: /root/.cache/pip/wheels/77/b7/47/b989889699f788f4fecdc534bfddbb16df9a9a51f4e431b04d
Successfully built pyqubo


In [ ]:
!dwave setup

Optionally install non-open-source packages and configure your environment.

Do you want to select non-open-source packages to install (y/n)? [y]: n

Creating the D-Wave configuration file.
Found existing configuration file: /root/.config/dwave/dwave.conf
Configuration file path [/root/.config/dwave/dwave.conf]: 
Profile (create new or choose from: prod): 
Input required, please try again.
Profile (create new or choose from: prod): 
Input required, please try again.
Profile (create new or choose from: prod): 
Aborted!


##Imports

In [ ]:
from __future__ import print_function, division
from google.colab import drive
drive.mount('/content/drive', force_remount = True)


!pip install -q git+https://github.com/tensorflow/examples.git

import glob
import pandas  as pd
import numpy   as np
import nibabel as nib
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras import datasets, layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from zipfile import ZipFile
from shutil import copyfile, copyfileobj
import gzip
from tensorflow_examples.models.pix2pix import pix2pix
from IPython.display import clear_output
from PIL import Image
import os
import matplotlib.image as mpimg
from scipy.ndimage import zoom


print("Version: ", tf.__version__)
print("Eager mode: ", tf.executing_eagerly())
print("GPU is", "available" if tf.config.experimental.list_physical_devices("GPU") else "NOT AVAILABLE")

Mounted at /content/drive
Version:  2.3.0
Eager mode:  True
GPU is NOT AVAILABLE


In [ ]:

import sys
sys.path.append("/content/drive/My Drive/Colab Notebooks/qboost")

from sklearn import preprocessing, metrics
from sklearn.ensemble import AdaBoostClassifier
from sklearn.datasets import load_breast_cancer, fetch_openml
from sklearn.impute import SimpleImputer
from dwave.system.samplers import DWaveSampler
from dwave.system.composites import EmbeddingComposite
from qboost import WeakClassifiers, QBoostClassifier, QboostPlus

##Classifier function

In [ ]:
def metric(y, y_pred):

    return metrics.accuracy_score(y, y_pred)


In [ ]:
def train_model(X_train, y_train, X_test, y_test, lmd):
    """
    Train qboost model

    :param X_train: train input
    :param y_train: train label
    :param X_test: test input
    :param y_test: test label
    :param lmd: lmbda to control regularization term
    :return:
    """
    NUM_READS = 3000
    NUM_WEAK_CLASSIFIERS = 60
    # lmd = 0.5
    TREE_DEPTH = 3

    # define sampler
    dwave_sampler = DWaveSampler(solver={'qpu': True})
    # sa_sampler = micro.dimod.SimulatedAnnealingSampler()
    emb_sampler = EmbeddingComposite(dwave_sampler)

    N_train = len(X_train)
    N_test = len(X_test)

    print("\n======================================")
    print("Train#: %d, Test: %d" %(N_train, N_test))
    print('Num weak classifiers:', NUM_WEAK_CLASSIFIERS)
    print('Tree depth:', TREE_DEPTH)


    # input: dataset X and labels y (in {+1, -1}

    # Preprocessing data
    # imputer = SimpleImputer()
    scaler = preprocessing.StandardScaler()     # standardize features
    normalizer = preprocessing.Normalizer()     # normalize samples

    # X = imputer.fit_transform(X)
    X_train = scaler.fit_transform(X_train)
    X_train = normalizer.fit_transform(X_train)

    # X_test = imputer.fit_transform(X_test)
    X_test = scaler.fit_transform(X_test)
    X_test = normalizer.fit_transform(X_test)

    ## Adaboost
    print('\nAdaboost')

    clf = AdaBoostClassifier(n_estimators=NUM_WEAK_CLASSIFIERS)

    # scores = cross_val_score(clf, X, y, cv=5, scoring='accuracy')
    print('fitting...')
    clf.fit(X_train, y_train)

    hypotheses_ada = clf.estimators_
    # clf.estimator_weights_ = np.random.uniform(0,1,size=NUM_WEAK_CLASSIFIERS)
    print('testing...')
    y_train_pred = clf.predict(X_train)
    y_test_pred = clf.predict(X_test)

    print('accu (train): %5.2f'%(metric(y_train, y_train_pred)))
    print('accu (test): %5.2f'%(metric(y_test, y_test_pred)))

    # Ensembles of Decision Tree
    print('\nDecision tree')

    clf2 = WeakClassifiers(n_estimators=NUM_WEAK_CLASSIFIERS, max_depth=TREE_DEPTH)
    clf2.fit(X_train, y_train)

    y_train_pred2 = clf2.predict(X_train)
    y_test_pred2 = clf2.predict(X_test)
    print(clf2.estimator_weights)

    print('accu (train): %5.2f' % (metric(y_train, y_train_pred2)))
    print('accu (test): %5.2f' % (metric(y_test, y_test_pred2)))

    # Ensembles of Decision Tree
    print('\nQBoost')

    DW_PARAMS = {'num_reads': NUM_READS,
                 'auto_scale': True,
                 # "answer_mode": "histogram",
                 'num_spin_reversal_transforms': 10,
                 # 'annealing_time': 10,
                 'postprocess': 'optimization',
                 }

    clf3 = QBoostClassifier(n_estimators=NUM_WEAK_CLASSIFIERS, max_depth=TREE_DEPTH)
    clf3.fit(X_train, y_train, emb_sampler, lmd=lmd, **DW_PARAMS)

    y_train_dw = clf3.predict(X_train)
    y_test_dw = clf3.predict(X_test)

    print(clf3.estimator_weights)

    print('accu (train): %5.2f' % (metric(y_train, y_train_dw)))
    print('accu (test): %5.2f' % (metric(y_test, y_test_dw)))


    # Ensembles of Decision Tree
    print('\nQBoostPlus')
    clf4 = QboostPlus([clf, clf2, clf3])
    clf4.fit(X_train, y_train, emb_sampler, lmd=lmd, **DW_PARAMS)
    y_train4 = clf4.predict(X_train)
    y_test4 = clf4.predict(X_test)
    print(clf4.estimator_weights)

    print('accu (train): %5.2f' % (metric(y_train, y_train4)))
    print('accu (test): %5.2f' % (metric(y_test, y_test4)))


    print("=============================================")
    print("Method \t Adaboost \t DecisionTree \t Qboost \t QboostIt")
    print("Train\t %5.2f \t\t %5.2f \t\t\t %5.2f \t\t %5.2f"% (metric(y_train, y_train_pred),
                                                               metric(y_train, y_train_pred2),
                                                               metric(y_train, y_train_dw),
                                                               metric(y_train, y_train4)))
    print("Test\t %5.2f \t\t %5.2f \t\t\t %5.2f \t\t %5.2f"% (metric(y_test, y_test_pred),
                                                              metric(y_test,y_test_pred2),
                                                              metric(y_test, y_test_dw),
                                                              metric(y_test, y_test4)))
    print("=============================================")

    # plt.subplot(211)
    # plt.bar(range(len(y_test)), y_test)
    # plt.subplot(212)
    # plt.bar(range(len(y_test)), y_test_dw)
    # plt.show()

    return


##Calling the classifier with relevant data

In [ ]:
# #del ncp_scan_list
# del ncp_scan_list
# #del normal_scan_list
# del normal_scan_list
# #del ncp_dataset_list
# #del normal_dataset_list



In [ ]:
n_train = 0.8
n_test = 0.2
path_ncp = '/content/drive/My Drive/Colab Notebooks/COVID19-1/NCP/'
path_cp = '/content/drive/My Drive/Colab Notebooks/CP/'
path_normal = '/content/drive/My Drive/Colab Notebooks/Ordered/'
path_ncp_save = '/content/drive/My Drive/Colab Notebooks/Masks_final/NCP'
path_cp_save = '/content/drive/My Drive/Colab Notebooks/Masks_final/CP'
path_normal_save = '/content/drive/My Drive/Colab Notebooks/Masks_final/Normal'
n_ncp = 30
n_cp = 20
n_normal = 50
masks_ncp = []
masks_cp = []
masks_normal = []
def mask_reader(path,n):
  mask_list = []
  for i,p in enumerate(os.listdir(path)):
    if i>=n:
      break
    mask = np.load(path + '/' + p)
    mask_list.append(mask)
    #print(mask.shape)
  return mask_list

masks_ncp = mask_reader(path_ncp_save,n_ncp)
masks_cp = mask_reader(path_cp_save,n_cp)
masks_normal = mask_reader(path_normal_save,n_normal)

patient_list = []

for mask in masks_ncp:
  patient_label = {}
  patient_label['mask'] = mask
  patient_label['label'] = 1
  patient_list.append(patient_label)
  del patient_label

del masks_ncp

for mask in masks_cp:
  patient_label = {}
  patient_label['mask'] = mask
  patient_label['label'] = 1
  patient_list.append(patient_label)
  del patient_label

del masks_cp

for mask in masks_normal:
  patient_label = {}
  patient_label['mask'] = mask
  patient_label['label'] = -1
  patient_list.append(patient_label)
  del patient_label

del masks_normal

print(len(patient_list))


100


In [ ]:
patient_list[5]['label']

1

In [ ]:
import random
random.shuffle(patient_list)


In [ ]:
for i in patient_list:
  print(i['label'])

-1
1
1
1
-1
-1
1
1
-1
-1
-1
-1
-1
-1
1
1
1
1
1
-1
-1
1
-1
1
1
-1
-1
-1
-1
1
-1
-1
1
1
1
1
1
-1
-1
1
1
1
-1
1
-1
1
1
-1
-1
1
1
-1
1
-1
-1
-1
-1
1
1
-1
-1
-1
-1
-1
-1
-1
-1
1
1
-1
1
1
1
-1
-1
1
1
1
-1
1
-1
1
1
1
-1
1
-1
1
1
1
1
-1
-1
1
1
-1
1
-1
-1
-1


In [ ]:
n_total = len(patient_list)
train_data = patient_list[:round(n_train*n_total)]
test_data = patient_list[round(n_train*n_total):]
print(len(train_data),len(test_data))

80 20


In [ ]:
print(type(train_data[0]['mask']),type(train_data[0]['label']))

<class 'numpy.ndarray'> <class 'int'>


In [ ]:
print(train_data[0]['mask'].shape,train_data[1]['mask'].shape)

(224, 224, 90) (224, 224, 90)


In [ ]:
# pad = np.zeros((224,224))
# max = 0
# for i in train_data:
#   if i['mask'].shape[2] > max:
#     max = i['mask'].shape[2]
# for i in test_data:
#   if i['mask'].shape[2] > max:
#     max = i['mask'].shape[2]

# print(max)

In [ ]:
# for i in train_data:
#   if i['mask'].shape[2] < max:
#     temp = np.zeros((224,224,max))
#     temp_size = round((max - i['mask'].shape[2])/2.0)
#     temp[:,:,temp_size:(temp_size+i['mask'].shape[2])] = i['mask']
#     i['mask'] = temp

In [ ]:
# for i in test_data:
#   if i['mask'].shape[2] < max:
#     temp = np.zeros((224,224,max))
#     temp_size = round((max - i['mask'].shape[2])/2.0)
#     temp[:,:,temp_size:(temp_size+i['mask'].shape[2])] = i['mask']
#     i['mask'] = temp

In [ ]:
Height = 224
Width = 224
for i in train_data:
  i['mask'] = np.reshape(i['mask'],(Height*Width*90,))
for i in test_data:
  i['mask'] = np.reshape(i['mask'],(Height*Width*90,))


In [ ]:
X_train = np.zeros((round(n_total*n_train),Height*Width*90))
Y_train = np.zeros((round(n_total*n_train),))
X_test = np.zeros((n_total - round(n_total*n_train),Height*Width*90))
Y_test = np.zeros((n_total - round(n_total*n_train),))


for idx,i in enumerate(train_data):
  X_train[idx,:] = i['mask']
  Y_train[idx] = i['label']

for idx,i in enumerate(test_data):
  X_test[idx,:] = i['mask']
  Y_test[idx] = i['label']


In [ ]:
print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)
print(Y_train,Y_test)

(80, 4515840) (80,)
(20, 4515840) (20,)
[-1.  1.  1.  1. -1. -1.  1.  1. -1. -1. -1. -1. -1. -1.  1.  1.  1.  1.
  1. -1. -1.  1. -1.  1.  1. -1. -1. -1. -1.  1. -1. -1.  1.  1.  1.  1.
  1. -1. -1.  1.  1.  1. -1.  1. -1.  1.  1. -1. -1.  1.  1. -1.  1. -1.
 -1. -1. -1.  1.  1. -1. -1. -1. -1. -1. -1. -1. -1.  1.  1. -1.  1.  1.
  1. -1. -1.  1.  1.  1. -1.  1.] [-1.  1.  1.  1. -1.  1. -1.  1.  1.  1.  1. -1. -1.  1.  1. -1.  1. -1.
 -1. -1.]


In [ ]:
clfs = train_model(X_train, Y_train, X_test, Y_test, 1.0)



Train#: 80, Test: 20
Num weak classifiers: 60
Tree depth: 3

Adaboost
fitting...
testing...
accu (train):  1.00
accu (test):  0.80

Decision tree
[23.02585093 23.02585093 23.02585093 23.02585093 23.02585093 23.02585093
 23.02585093 23.02585093 23.02585093 23.02585093 23.02585093 23.02585093
 23.02585093 23.02585093 23.02585093 23.02585093 23.02585093 23.02585093
 23.02585093 23.02585093 23.02585093 23.02585093 23.02585093 23.02585093
 23.02585093 23.02585093 23.02585093 23.02585093 23.02585093 23.02585093
 23.02585093 23.02585093 23.02585093 23.02585093 23.02585093 23.02585093
 23.02585093 23.02585093 23.02585093 23.02585093 23.02585093 23.02585093
 23.02585093 23.02585093 23.02585093 23.02585093 23.02585093 23.02585093
 23.02585093 23.02585093 23.02585093 23.02585093 23.02585093 23.02585093
 23.02585093 23.02585093 23.02585093 23.02585093 23.02585093 23.02585093]
accu (train):  1.00
accu (test):  0.70

QBoost
[1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
